<a href="https://colab.research.google.com/github/dayaniravi123/Machine-Learning-Projects/blob/main/NLP_Problem_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text

In [99]:
import tensorflow as tf
import pandas as pd
import numpy as np

import tensorflow_text as tf_text
import re
import string
import nltk
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

In [60]:
dataset = pd.read_csv('/content/drive/MyDrive/Machine Learning - Tensorflow /NLP - IMDB/IMDB Dataset.csv')

dataset

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [76]:
tokenizer = tf_text.UnicodeScriptTokenizer()

db = []

for i in range(100):
  temp = tokenizer.tokenize(dataset['review'][i])
  db.append(temp)

In [93]:
temp = dataset['review'][0]
print(temp)


temp = [c for c in temp if c not in string.punctuation]
temp = ''.join(temp)

stemmer = nltk.porter.PorterStemmer()
temp = ' '.join([stemmer.stem(word) for word in temp.split()])
    
print(temp)


One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [96]:
NLP_model = Pipeline([
                      ('counter', CountVectorizer(stop_words='english')),
                      ('tf-idf', TfidfTransformer()),
                      ('svm', SGDClassifier(loss= 'hinge', penalty='l2', alpha= 1e-3, random_state= 42))
                    ])

NLP_model.fit(dataset['review'], dataset['sentiment'])

Pipeline(steps=[('counter', CountVectorizer(stop_words='english')),
                ('tf-idf', TfidfTransformer()),
                ('svm', SGDClassifier(alpha=0.001, random_state=42))])

In [97]:
prediction = NLP_model.predict(dataset['review'])

np.mean(prediction == dataset['sentiment'])

0.84622

In [100]:
NLP_model = Pipeline([
                      ('counter', CountVectorizer(stop_words='english')),
                      ('tf-idf', TfidfTransformer()),
                      ('NNClassifier', MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 3), random_state=1))
                    ])

NLP_model.fit(dataset['review'], dataset['sentiment'])



Pipeline(steps=[('counter', CountVectorizer(stop_words='english')),
                ('tf-idf', TfidfTransformer()),
                ('NNClassifier',
                 MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 3),
                               random_state=1))])

In [101]:
prediction = NLP_model.predict(dataset['review'])

np.mean(prediction == dataset['sentiment'])

1.0